## Accelerated Training of Transformer Models
This notebook demonstrates using [ONNX Runtime](https://cloudblogs.microsoft.com/opensource/2020/05/19/announcing-support-for-accelerated-training-with-onnx-runtime/) to accelerate the training of a sequence-to-sequence model. It uses the implementation available at [PyTorch tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html) which uses the [nn.Transformer](https://pytorch.org/docs/master/nn.html?highlight=nn%20transformer#torch.nn.Transformer) module. This notebook is available at [https://github.com/skaarthik/onnxruntime-training-databricks](https://github.com/skaarthik/onnxruntime-training-databricks). Additional examples on pretraining and finetuning involving BERT and GPT-2 are available at [https://github.com/microsoft/onnxruntime-training-examples](https://github.com/microsoft/onnxruntime-training-examples).

Notes: 
* Custom containers are not supported on ML Runtime in Databricks. This demo needs ML Runtime and GPUs. As an alternative to packaging dependencies in a customer container, refer to the `Prerequisites` section below to prepare Databricks environment for this demo.
* `%load` and `%run` commands are not supported in Databricks notebooks (see [Databricks Notebooks documentation](https://docs.databricks.com/notebooks/notebooks-use.html#run-a-notebook-from-another-notebook) for details). Due to this limitation, this notebook has the Python scripts included instead of loading directly from the GitHub repo.

####Prerequisites for the demo
Run the following commands to install the following packages to the Conda environment you will be using:
* pip install torchtext
* pip install onnx
* pip install cerberus
* pip install https://onnxtraining.blob.core.windows.net/ort-databricks-demo/onnxruntime_gpu-1.5.1-cp37-cp37m-linux_x86_64.whl

####Define the model
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#define-the-model for details.

In [4]:
import math
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, src):
        sz = src.size(0)
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask == None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

####Load and batch data
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#load-and-batch-data for details.

In [6]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda")

/databricks/python/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
 warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/databricks/python/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
 warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)

####Functions to generate input and target sequence
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#functions-to-generate-input-and-target-sequence for details.

In [8]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
  
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)  

####Initiate an instance
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#initiate-an-instance for details.

In [10]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
lr = 0.001

def calculate_loss(output, targets):
    output = output.view(-1, len(TEXT.vocab.stoi))
    return criterion(output, targets) 

In [11]:
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = lr
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
metric_prefix = 'baseline'  

In [14]:
import mlflow
def log_metrics(epoch, batch, train_data_len, bptt, lr, elapsed, log_interval, cur_loss, log_prefix):
  print('| epoch {:3d} | {:5d}/{:5d} batches | '
        'lr {:02.3f} | ms/batch {:5.2f} | '
        'loss {:5.2f} | ppl {:8.2f}'.format(
          epoch, batch, train_data_len // bptt, lr,
          elapsed * 1000 / log_interval,
          cur_loss, math.exp(cur_loss)))
  mlflow.log_metric(log_prefix + '_milliseconds/batch', elapsed * 1000 / log_interval, step=batch)

####Define `train` and `evaluate` methods
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#run-the-model for details.

In [16]:
import time
def train(epoch, accelerate_using_ort):
    if not accelerate_using_ort:
      model.train() # Turn on the train mode
      
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        
        if not accelerate_using_ort:
          optimizer.zero_grad()
          output = model(data)
          loss = calculate_loss(output, targets)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
          optimizer.step()
          current_learning_rate = learning_rate
        else:
          loss, output = trainer.train_step(data, targets)
          current_learning_rate = learning_rate

        total_loss += loss.item()
        log_interval = 50

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time

            log_metrics(epoch, 
                        batch, 
                        len(train_data),
                        bptt, 
                        current_learning_rate,
                        elapsed,
                        log_interval,
                        cur_loss,
                        metric_prefix)
            
            total_loss = 0
            start_time = time.time()           

In [17]:
def evaluate(eval_model, data_source, accelerate_using_ort):
    if not accelerate_using_ort:
      eval_model.eval() # Turn on the evaluation mode
      
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if not accelerate_using_ort:
              output = eval_model(data)
              output_flat = output.view(-1, ntokens)
              total_loss += len(data) * criterion(output_flat, targets).item()
            else:
              loss, outputs = trainer.eval_step(data, targets)
              total_loss += len(data) * loss.item()              
            
    return total_loss / (len(data_source) - 1)

####Train the model
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#run-the-model for details.

In [19]:
def train_model(accelerate_using_ort=False):
  best_val_loss = float("inf")
  epochs = 3 # The number of epochs
  best_model = None

  for epoch in range(1, epochs + 1):
      epoch_start_time = time.time()
      
      train(epoch, accelerate_using_ort)
        
      val_loss = evaluate(model, val_data, accelerate_using_ort)
      print('-' * 89)
      print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                       val_loss, math.exp(val_loss)))
      print('-' * 89)

      if val_loss < best_val_loss:
          best_val_loss = val_loss
          best_model = model
  return best_model

In [20]:
baseline_model = train_model(accelerate_using_ort=False)

| epoch 1 | 50/ 2981 batches | lr 0.001 | ms/batch 16.92 | loss 7.73 | ppl 2265.90
 epoch 1 | 100/ 2981 batches | lr 0.001 | ms/batch 12.12 | loss 6.70 | ppl 811.65
 epoch 1 | 150/ 2981 batches | lr 0.001 | ms/batch 12.47 | loss 6.50 | ppl 667.39
 epoch 1 | 200/ 2981 batches | lr 0.001 | ms/batch 12.30 | loss 6.35 | ppl 572.15
 epoch 1 | 250/ 2981 batches | lr 0.001 | ms/batch 12.40 | loss 6.26 | ppl 523.21
 epoch 1 | 300/ 2981 batches | lr 0.001 | ms/batch 12.14 | loss 6.22 | ppl 503.98
 epoch 1 | 350/ 2981 batches | lr 0.001 | ms/batch 12.13 | loss 6.09 | ppl 443.11
 epoch 1 | 400/ 2981 batches | lr 0.001 | ms/batch 12.12 | loss 6.07 | ppl 433.20
 epoch 1 | 450/ 2981 batches | lr 0.001 | ms/batch 12.23 | loss 5.93 | ppl 377.42
 epoch 1 | 500/ 2981 batches | lr 0.001 | ms/batch 12.22 | loss 6.02 | ppl 409.71
 epoch 1 | 550/ 2981 batches | lr 0.001 | ms/batch 12.43 | loss 5.88 | ppl 359.41
 epoch 1 | 600/ 2981 batches | lr 0.001 | ms/batch 12.28 | loss 5.81 | ppl 333.39
 epoch 1 | 650/ 2981 batches | lr 0.001 | ms/batch 12.18 | loss 5.93 | ppl 377.16
 epoch 1 | 700/ 2981 batches | lr 0.001 | ms/batch 12.28 | loss 5.84 | ppl 343.27
 epoch 1 | 750/ 2981 batches | lr 0.001 | ms/batch 12.22 | loss 5.78 | ppl 324.97
 epoch 1 | 800/ 2981 batches | lr 0.001 | ms/batch 12.37 | loss 5.88 | ppl 358.71
 epoch 1 | 850/ 2981 batches | lr 0.001 | ms/batch 12.23 | loss 5.78 | ppl 322.77
 epoch 1 | 900/ 2981 batches | lr 0.001 | ms/batch 12.20 | loss 5.74 | ppl 309.91
 epoch 1 | 950/ 2981 batches | lr 0.001 | ms/batch 12.47 | loss 5.78 | ppl 323.99
 epoch 1 | 1000/ 2981 batches | lr 0.001 | ms/batch 12.22 | loss 5.72 | ppl 303.70
 epoch 1 | 1050/ 2981 batches | lr 0.001 | ms/batch 12.25 | loss 5.77 | ppl 321.75
 epoch 1 | 1100/ 2981 batches | lr 0.001 | ms/batch 12.25 | loss 5.69 | ppl 295.88
 epoch 1 | 1150/ 2981 batches | lr 0.001 | ms/batch 12.21 | loss 5.69 | ppl 295.65
 epoch 1 | 1200/ 2981 batches | lr 0.001 | ms/batch 12.30 | loss 5.79 | ppl 328.18
 epoch 1 | 1250/ 2981 batches | lr 0.001 | ms/batch 12.18 | loss 5.75 | ppl 312.72
 epoch 1 | 1300/ 2981 batches | lr 0.001 | ms/batch 12.21 | loss 5.81 | ppl 334.17
 epoch 1 | 1350/ 2981 batches | lr 0.001 | ms/batch 12.29 | loss 5.69 | ppl 295.12
 epoch 1 | 1400/ 2981 batches | lr 0.001 | ms/batch 12.29 | loss 5.65 | ppl 283.23
 epoch 1 | 1450/ 2981 batches | lr 0.001 | ms/batch 12.25 | loss 5.75 | ppl 313.70
 epoch 1 | 1500/ 2981 batches | lr 0.001 | ms/batch 12.45 | loss 5.78 | ppl 322.31
 epoch 1 | 1550/ 2981 batches | lr 0.001 | ms/batch 12.29 | loss 5.72 | ppl 304.06
 epoch 1 | 1600/ 2981 batches | lr 0.001 | ms/batch 12.30 | loss 5.65 | ppl 284.63
 epoch 1 | 1650/ 2981 batches | lr 0.001 | ms/batch 12.28 | loss 5.59 | ppl 268.38
 epoch 1 | 1700/ 2981 batches | lr 0.001 | ms/batch 12.30 | loss 5.61 | ppl 274.10
 epoch 1 | 1750/ 2981 batches | lr 0.001 | ms/batch 12.29 | loss 5.60 | ppl 269.77
 epoch 1 | 1800/ 2981 batches | lr 0.001 | ms/batch 12.22 | loss 5.63 | ppl 277.75
 epoch 1 | 1850/ 2981 batches | lr 0.001 | ms/batch 12.48 | loss 5.68 | ppl 292.29
 epoch 1 | 1900/ 2981 batches | lr 0.001 | ms/batch 12.35 | loss 5.61 | ppl 273.17
 epoch 1 | 1950/ 2981 batches | lr 0.001 | ms/batch 12.20 | loss 5.58 | ppl 264.36
 epoch 1 | 2000/ 2981 batches | lr 0.001 | ms/batch 12.14 | loss 5.58 | ppl 265.90
 epoch 1 | 2050/ 2981 batches | lr 0.001 | ms/batch 12.31 | loss 5.57 | ppl 262.43
 epoch 1 | 2100/ 2981 batches | lr 0.001 | ms/batch 12.34 | loss 5.47 | ppl 236.78
 epoch 1 | 2150/ 2981 batches | lr 0.001 | ms/batch 12.52 | loss 5.41 | ppl 224.60
 epoch 1 | 2200/ 2981 batches | lr 0.001 | ms/batch 12.54 | loss 5.46 | ppl 235.69
 epoch 1 | 2250/ 2981 batches | lr 0.001 | ms/batch 12.43 | loss 5.47 | ppl 237.23
 epoch 1 | 2300/ 2981 batches | lr 0.001 | ms/batch 12.34 | loss 5.56 | ppl 260.50
 epoch 1 | 2350/ 2981 batches | lr 0.001 | ms/batch 12.54 | loss 5.63 | ppl 277.49
 epoch 1 | 2400/ 2981 batches | lr 0.001 | ms/batch 12.45 | loss 5.55 | ppl 257.84
 epoch 1 | 2450/ 2981 batches | lr

####Evaluate the model with the test dataset
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#evaluate-the-model-with-the-test-dataset for details.

In [22]:
test_loss = evaluate(baseline_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

####Define `train_in_ort` method
Refer to https://github.com/microsoft/onnxruntime-training-examples/tree/master/getting-started for details on changes needed to accelerate a PyTorch implementation using ONNX Runtime (ORT).

In [24]:
from onnxruntime.training import ORTTrainer, optim

model_description = {'inputs':  [('src', ['bptt', 'batch_size']),
                                 ('label', ['bptt_x_batch_size'])],
                     'outputs': [('loss', [], True),
                                 ('output', ['bptt', 'batch_size', ntokens])]}
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = lr
optimizer_config = optim.AdamConfig(lr=learning_rate)
trainer = ORTTrainer(model,               # model
                     model_description,   # model description
                     optimizer_config,    # optimizer configuration
                     calculate_loss)      # loss function
metric_prefix = 'onnxruntime'

In [25]:
ort_model = train_model(accelerate_using_ort=True)

####Evaluate the model with the test dataset
Refer to https://pytorch.org/tutorials/beginner/transformer_tutorial.html#evaluate-the-model-with-the-test-dataset for details.

In [27]:
test_loss = evaluate(ort_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)